In [ ]:
def trading_strategy(data, strategy_type):
    total_profit = 0
    total_loss = 0
    wins = 0
    losses = 0
    trades = 0
    bought = False
    max_profit = 0
    max_loss = 0
    max_holding_period = 0
    total_holding_period = 0
    entry_price = 0
    entry_price_index = 0  # 매수 시점을 기록하기 위한 변수
    data['Adj Open'] = data['Open'] * data['Adj Close'] / data['Close']  # 수정 시가
    
    # 전략에 따른 처리
    for i in range(1, len(data)):
        if strategy_type == 1:  # 첫 신호 매매
            if not bought and data['Pre'].iloc[i] == 1 and data['Pre'].iloc[i-1] == 0:  # 매수 신호
                entry_price_index = i  # 매수 시점 기록
                bought = True
            elif bought and data['Pre'].iloc[i] == 0 and data['Pre'].iloc[i-1] == 1:  # 매도 신호
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]  # 전날 시가와의 차이
                holding_period = i - entry_price_index  # 보유 기간 계산
                total_holding_period += holding_period  # 총 보유 기간
                max_holding_period = max(max_holding_period, holding_period)  # 최대 보유 기간 갱신
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass
                trades += 1
                bought = False
            elif bought:  # hold
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass 
            else:  # 매도 ~ 매수
                pass  
            
    # 마지막에 주식을 보유하고 있다면 종가로 청산
    if bought:
        result = data['Adj Close'].iloc[-1] - data['Adj Open'].iloc[-1]
        holding_period = len(data) - entry_price_index + 1  # 마지막까지 보유했을 때의 기간 계산
        total_holding_period += holding_period
        max_holding_period = max(max_holding_period, holding_period)  # 최대 보유 기간 갱신
        if result > 0:
            total_profit += result
            wins += 1
            max_profit = max(max_profit, result)
        elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
        else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
            pass
        trades += 1


    # 성과 지표 계산
    average_profit_per_winning_trade = total_profit / wins if wins > 0 else float('inf')
    average_loss_per_losing_trade = total_loss / losses if losses > 0 else float('inf')
    payoff_ratio = average_profit_per_winning_trade / average_loss_per_losing_trade if average_loss_per_losing_trade != 0 else float('inf')
    profit_factor = total_profit / total_loss if total_loss != 0 else float('inf')
    win_rate = (wins / trades) * 100 if trades > 0 else float('inf')

    # 지표 반환
    return {
        "Total Profit": total_profit,  # 누적 이익
        "Total Loss": total_loss,  # 누적 손실
        "Payoff Ratio": payoff_ratio,  # 지급 비율
        "Profit Factor": profit_factor,  # 이익 계수
        "Win Rate": win_rate,  # 승률
        "Total Trades": trades,  # 총 거래 수
        "Max Holding Period": max_holding_period,  # 최대 홀딩 기간
        "Max Profit": max_profit,  # 최대 이익 
        "Max Loss": max_loss,  # 최대손실
    }                                 

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import talib as ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (Windows의 경우 기본적으로 'Malgun Gothic'을 사용)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 종목 리스트 정의
tickers = [
    '005930.KS', '005935.KS', '000660.KS', '005387.KS', '005385.KS', '005380.KS', '005389.KS', '068270.KS',
    '000270.KS', '105560.KS', '055550.KS', '035420.KS', '005490.KS', '02826K.KS', '028260.KS', '051910.KS',
    '051915.KS', '006400.KS', '006405.KS', '012330.KS', '086790.KS', '035720.KS', '066575.KS', '066570.KS',
    '032830.KS', '012450.KS', '000815.KS', '000810.KS', '009540.KS', '011200.KS', '003555.KS', '003550.KS',
    '015760.KS', '017670.KS', '034020.KS', '018260.KS', '033780.KS', '024110.KS', '010130.KS', '009150.KS'
]

# 전략별 결과를 저장할 리스트 초기화
results_test_all = {i: [] for i in range(1, 7)}  # 전략 1 ~ 6

In [ ]:
def trading_strategy(data, strategy_type):
    total_profit = 0
    total_loss = 0
    wins = 0
    losses = 0
    trades = 0
    bought = False
    max_profit = 0
    max_loss = 0
    max_holding_period = 0
    total_holding_period = 0
    entry_price = 0
    entry_price_index = 0  # 매수 시점을 기록하기 위한 변수
    data['Adj Open'] = data['Open'] * data['Adj Close'] / data['Close']  # 수정 시가

    # 전략에 따른 처리           
    for i in range(1, len(data)):
        if strategy_type == 1:  # 첫 신호 매매
            if not bought and data['Pre'].iloc[i] == 1 and data['Pre'].iloc[i-1] == 0:  # 매수 신호
                entry_price_index = i  # 매수 시점 기록
                bought = True
            elif bought and data['Pre'].iloc[i] == 0 and data['Pre'].iloc[i-1] == 1:  # 매도 신호
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]  # 전날 시가와의 차이
                holding_period = i - entry_price_index  # 보유 기간 계산
                total_holding_period += holding_period  # 총 보유 기간
                max_holding_period = max(max_holding_period, holding_period)  # 최대 보유 기간 갱신
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass
                trades += 1
                bought = False
            elif bought:  # hold
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass 
            else:  # 매도 ~ 매수
                pass 

        elif strategy_type == 2:  # 두 번 연속 신호 매매
            if i >= 2 and not bought and data['Pre'].iloc[i] == 1 and data['Pre'].iloc[i-1] == 1 and data['Pre'].iloc[i-2] == 0:
                entry_price_index = i
                bought = True
            elif i >= 2 and bought and data['Pre'].iloc[i] == 0 and data['Pre'].iloc[i-1] == 0 and data['Pre'].iloc[i-2] == 1:
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                holding_period = i - entry_price_index
                total_holding_period += holding_period
                max_holding_period = max(max_holding_period, holding_period)
                if result > 0:
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                trades += 1
                bought = False
            elif bought:  # hold
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass 
            else:  # 매도 ~ 매수
                pass 

        elif strategy_type == 3:  # 세 번 연속 신호 매매
            if i >= 3 and not bought and data['Pre'].iloc[i] == 1 and data['Pre'].iloc[i-1] == 1 and data['Pre'].iloc[i-2] == 1 and data['Pre'].iloc[i-3] == 0:
                entry_price_index = i
                bought = True
            elif i >= 3 and bought and data['Pre'].iloc[i] == 0 and data['Pre'].iloc[i-1] == 0 and data['Pre'].iloc[i-2] == 0 and data['Pre'].iloc[i-3] == 1:
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                holding_period = i - entry_price_index
                total_holding_period += holding_period
                max_holding_period = max(max_holding_period, holding_period)
                if result > 0:
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                trades += 1
                bought = False
            elif bought:  # hold
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass 
            else:  # 매도 ~ 매수
                pass 

        elif strategy_type == 4:  # 세 번 연속 상승, 두 번 연속 하락
            if i >= 3 and not bought and data['Pre'].iloc[i] == 1 and data['Pre'].iloc[i-1] == 1 and data['Pre'].iloc[i-2] == 1 and data['Pre'].iloc[i-3] == 0:
                entry_price_index = i
                bought = True
            elif i >= 2 and bought and data['Pre'].iloc[i] == 0 and data['Pre'].iloc[i-1] == 0 and data['Pre'].iloc[i-2] == 1:
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                holding_period = i - entry_price_index
                total_holding_period += holding_period
                max_holding_period = max(max_holding_period, holding_period)
                if result > 0:
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                trades += 1
                bought = False
            elif bought:  # hold
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass 
            else:  # 매도 ~ 매수
                pass 
            
        elif strategy_type == 5:  # 네 번 연속 상승, 세 번 연속 하락
            if i >= 4 and not bought and data['Pre'].iloc[i] == 1 and data['Pre'].iloc[i-1] == 1 and data['Pre'].iloc[i-2] == 1 and data['Pre'].iloc[i-3] == 1 and data['Pre'].iloc[i-4] == 0:
                entry_price_index = i
                bought = True
            elif i >= 3 and bought and data['Pre'].iloc[i] == 0 and data['Pre'].iloc[i-1] == 0 and data['Pre'].iloc[i-2] == 0 and data['Pre'].iloc[i-3] == 1:
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                holding_period = i - entry_price_index
                total_holding_period += holding_period
                max_holding_period = max(max_holding_period, holding_period)
                if result > 0:
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                trades += 1
                bought = False
            elif bought:  # hold
                result = data['Adj Open'].iloc[i] - data['Adj Open'].iloc[i-1]
                if result > 0:  # 수익
                    total_profit += result
                    wins += 1
                    max_profit = max(max_profit, result)
                elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
                else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
                    pass 
            else:  # 매도 ~ 매수
                pass 

    # 마지막에 주식을 보유하고 있다면 종가로 청산
    if bought:
        result = data['Adj Close'].iloc[-1] - data['Adj Open'].iloc[-1]
        holding_period = len(data) - entry_price_index + 1  # 마지막까지 보유했을 때의 기간 계산
        total_holding_period += holding_period
        max_holding_period = max(max_holding_period, holding_period)  # 최대 보유 기간 갱신
        if result > 0:
            total_profit += result
            wins += 1
            max_profit = max(max_profit, result)
        elif result < 0:  # 손실
                    total_loss += abs(result)
                    losses += 1
                    max_loss = max(max_loss, abs(result))
        else:  # 수익 = 0(전날 시가와 현재 시가가 같을 때)
            pass
        trades += 1


    # 성과 지표 계산
    average_profit_per_winning_trade = total_profit / wins if wins > 0 else float('inf')
    average_loss_per_losing_trade = total_loss / losses if losses > 0 else float('inf')
    payoff_ratio = average_profit_per_winning_trade / average_loss_per_losing_trade if average_loss_per_losing_trade != 0 else float('inf')
    profit_factor = total_profit / total_loss if total_loss != 0 else float('inf')
    win_rate = (wins / trades) * 100 if trades > 0 else float('inf')

    # 지표 반환
    return {
        "Total Profit": total_profit,  # 누적 이익
        "Total Loss": total_loss,  # 누적 손실
        "Payoff Ratio": payoff_ratio,  # 지급 비율
        "Profit Factor": profit_factor,  # 이익 계수
        "Win Rate": win_rate,  # 승률
        "Total Trades": trades,  # 총 거래 수
        "Max Holding Period": max_holding_period,  # 최대 홀딩 기간
        "Max Profit": max_profit,  # 최대 이익 
        "Max Loss": max_loss,  # 최대손실
    }                                 


In [ ]:
# 각 종목에 대해 모델 실행 및 결과 저장
for ticker in tickers:
    # 주식 데이터를 가져옴
    df = yf.Ticker(ticker).history(start='2009-01-01', end='2024-08-01', auto_adjust=False)
    
    # 타임존 제거
    df.index = df.index.tz_localize(None)

    # 필요한 기술적 지표들 추가
    df['RSI'] = ta.RSI(df['Adj Close'])
    df['Stochastic_Fast_K'], df['Stochastic_Fast_D'] = ta.STOCHF(df['High'], df['Low'], df['Adj Close'])
    df['Stochastic_Slow_K'], df['Stochastic_Slow_D'] = ta.STOCH(df['High'], df['Low'], df['Adj Close'])
    df['CCI'] = ta.CCI(df['High'], df['Low'], df['Adj Close'])
    df['OBV'] = ta.OBV(df['Adj Close'], df['Volume'])
    df['MFI'] = ta.MFI(df['High'], df['Low'], df['Adj Close'], df['Volume'])
    df['Aroon_Up'], df['Aroon_Down'] = ta.AROON(df['High'], df['Low'])
    df['Stochastic_RSI'] = ta.STOCHRSI(df['Adj Close'])[0]
    df['Ultimate_Oscillator'] = ta.ULTOSC(df['High'], df['Low'], df['Adj Close'])
    df['ADX'] = ta.ADX(df['High'], df['Low'], df['Adj Close'])
    df['ADXR'] = ta.ADXR(df['High'], df['Low'], df['Adj Close'])

    # 2010년 이후 데이터만 사용
    df = df[df.index >= '2010-01-01']

    # NaN이 포함되지 않은 데이터만 사용
    df = df.dropna()

    # 학습 데이터와 테스트 데이터로 분리
    train_data_final = df[(df.index >= '2010-01-01') & (df.index <= '2021-08-31')].copy()
    test_data = df[(df.index >= '2021-09-01') & (df.index <= '2024-07-31')].copy()

    # 'Act' 변수 추가 - 다음 거래일의 수정 종가가 전날 수정 종가보다 높은 경우 1, 낮은 경우 0
    train_data_final.loc[:, 'Act'] = np.where(train_data_final['Adj Close'].shift(-1) < train_data_final['Adj Close'], 1, 0)
    test_data.loc[:, 'Act'] = np.where(test_data['Adj Close'].shift(-1) < test_data['Adj Close'], 1, 0)

    # 입력 변수와 타겟 변수 설정
    X_train = train_data_final[['RSI', 'Stochastic_Fast_K', 'Stochastic_Fast_D', 'Stochastic_Slow_K', 'Stochastic_Slow_D', 
                                'CCI', 'OBV', 'MFI', 'Aroon_Up', 'Aroon_Down', 'Stochastic_RSI', 'Ultimate_Oscillator', 
                                'ADX', 'ADXR']]
    y_train = train_data_final['Act']

    X_test = test_data[['RSI', 'Stochastic_Fast_K', 'Stochastic_Fast_D', 'Stochastic_Slow_K', 'Stochastic_Slow_D', 
                        'CCI', 'OBV', 'MFI', 'Aroon_Up', 'Aroon_Down', 'Stochastic_RSI', 'Ultimate_Oscillator', 
                        'ADX', 'ADXR']]
    y_test = test_data['Act']

    optimized_models = {}

    # XGBoost 기본 모델 학습
    optimized_models["XGBoost"] = xgb.XGBClassifier()
    optimized_models["XGBoost"].fit(X_train, y_train)

    # LightGBM 기본 모델 학습
    optimized_models["LightGBM"] = lgb.LGBMClassifier()
    optimized_models["LightGBM"].fit(X_train, y_train)

    # Random Forest 기본 모델 학습
    optimized_models["Random Forest"] = RandomForestClassifier()
    optimized_models["Random Forest"].fit(X_train, y_train)

    # Decision Tree 기본 모델 학습
    optimized_models["Decision Tree"] = DecisionTreeClassifier()
    optimized_models["Decision Tree"].fit(X_train, y_train)

    # 각 전략별 결과 저장 (Naive Bayes와 Logistic Regression 제외)
    for name, model in optimized_models.items():
        # 테스트 데이터에 대한 예측 및 평가
        test_data['Pre'] = model.predict(X_test)

        # Pre와 Act 결과를 CSV로 저장
        pre_act_results = test_data[['Pre', 'Act']]
        pre_act_results.to_csv(f"{ticker}_{name}_pre_act_results.csv", index=False)

        for strategy_type in range(1, 7):  # 1 ~ 6번 전략 (6번이 Buy and Hold 전략)
            test_metrics = trading_strategy(test_data, strategy_type)
            results_test_all[strategy_type].append([
                ticker, 
                name, 
                test_metrics['Total Profit'], 
                test_metrics['Total Loss'], 
                test_metrics['Payoff Ratio'], 
                test_metrics['Profit Factor'], 
                test_metrics['Win Rate'], 
                test_metrics['Total Trades'],
                test_metrics['Max Holding Period'],
                test_metrics['Max Profit'],
                test_metrics['Max Loss'],
            ])

# 결과 저장 및 시각화 부분
for strategy_type in range(1, 7):
    df_results = pd.DataFrame(results_test_all[strategy_type], 
                              columns=['Ticker', 'Model', 'Total Profit', 'Total Loss', 
                                       'Payoff Ratio', 'Profit Factor', 'Win Rate', 'Total Trades', 
                                       'Max Holding Period', 'Max Profit', 'Max Loss',])

    # 결과 CSV로 저장
    df_results.to_csv(f"strategy_{strategy_type}_results.csv", index=False)

    # 시각화 및 결과 출력 (기존 코드 유지)
    plt.figure(figsize=(14, 8))
    sns.boxplot(x='Model', y='Total Profit', data=df_results, palette='Set3')
    sns.stripplot(x='Model', y='Total Profit', data=df_results, dodge=True, jitter=True, alpha=0.5, color='black', marker='o')
    
    plt.title(f'전략 {strategy_type}별 종목 및 모델에 따른 누적 수익 분포')
    plt.xlabel('모델')
    plt.ylabel('누적 수익 (원)')
    plt.xticks(rotation=45)
    plt.show()

    # 결과 표 출력
    print(f"\n전략 {strategy_type} 결과:\n")
    print(df_results.to_string(index=False)) 